In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

In [13]:
MAX_TRAINING = 100 #25000
MAX_VALIDATION = 50 #5000
MAX_TESTING = 40 #4000
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 15, 1024
architecture = 'LeNet5'
label, keywords = 'milestone-experiments2', 'benchmark_testing'

In [4]:
#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

# Keeping the data between 0 and 1
x_train_max, x_test_max = x_train / 255.0, x_test / 255.0
y_train_max = to_categorical(y_train, num_classes=10)
y_test_max = to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*1.5)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_train_max, y_train_max, train_cutoff+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

170500096/170498071 [==============================] - 4s 0us/step
Number of Training Samples: X=100, Y=100
Number of Validation Samples: X=50, Y=50
Number of Test Samples: X=40, Y=40


In [5]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [16]:
def model_extractor(activation_func):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

    # #Instantiating Layer 2
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))

    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(84, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [14]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
    if label == 'RAdam':
        model.save("../../saved_models/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
    else: 
        model.save("../../saved_models/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [17]:
testing_optimizers = ['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh', 'relu', 'selu']

for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.005), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)


        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

Train on 100 samples
Epoch 1/15
100/100 [==============================] - 2s 19ms/sample - loss: 2.3181 - acc: 0.1000 - mean_absolute_error: 0.1800 - mean_squared_error: 0.0903
Epoch 2/15
100/100 [==============================] - 0s 182us/sample - loss: 2.3137 - acc: 0.1100 - mean_absolute_error: 0.1800 - mean_squared_error: 0.0902
Epoch 3/15
100/100 [==============================] - 0s 172us/sample - loss: 2.3094 - acc: 0.1200 - mean_absolute_error: 0.1799 - mean_squared_error: 0.0901
Epoch 4/15
100/100 [==============================] - 0s 177us/sample - loss: 2.3054 - acc: 0.1200 - mean_absolute_error: 0.1798 - mean_squared_error: 0.0900
Epoch 5/15
100/100 [==============================] - 0s 179us/sample - loss: 2.3015 - acc: 0.1200 - mean_absolute_error: 0.1797 - mean_squared_error: 0.0900
Epoch 6/15
100/100 [==============================] - 0s 178us/sample - loss: 2.2978 - acc: 0.1200 - mean_absolute_error: 0.1797 - mean_squared_error: 0.0899
Epoch 7/15
100/100 [============

Train on 100 samples
Epoch 1/15
100/100 [==============================] - 1s 5ms/sample - loss: 2.3078 - acc: 0.0900 - mean_absolute_error: 0.1799 - mean_squared_error: 0.0901
Epoch 2/15
100/100 [==============================] - 0s 547us/sample - loss: 2.2134 - acc: 0.1700 - mean_absolute_error: 0.1775 - mean_squared_error: 0.0881
Epoch 3/15
100/100 [==============================] - 0s 237us/sample - loss: 2.1604 - acc: 0.1600 - mean_absolute_error: 0.1752 - mean_squared_error: 0.0868
Epoch 4/15
100/100 [==============================] - 0s 188us/sample - loss: 2.1132 - acc: 0.2400 - mean_absolute_error: 0.1734 - mean_squared_error: 0.0855
Epoch 5/15
100/100 [==============================] - 0s 267us/sample - loss: 2.0599 - acc: 0.3300 - mean_absolute_error: 0.1719 - mean_squared_error: 0.0840
Epoch 6/15
100/100 [==============================] - 0s 198us/sample - loss: 2.0008 - acc: 0.3800 - mean_absolute_error: 0.1702 - mean_squared_error: 0.0824
Epoch 7/15
100/100 [=============

Epoch 3/15
100/100 [==============================] - 0s 188us/sample - loss: 2.1778 - acc: 0.2500 - mean_absolute_error: 0.1768 - mean_squared_error: 0.0873
Epoch 4/15
100/100 [==============================] - 0s 167us/sample - loss: 2.1333 - acc: 0.1900 - mean_absolute_error: 0.1752 - mean_squared_error: 0.0862
Epoch 5/15
100/100 [==============================] - 0s 173us/sample - loss: 2.0951 - acc: 0.2800 - mean_absolute_error: 0.1740 - mean_squared_error: 0.0854
Epoch 6/15
100/100 [==============================] - 0s 166us/sample - loss: 2.0785 - acc: 0.2400 - mean_absolute_error: 0.1721 - mean_squared_error: 0.0846
Epoch 7/15
100/100 [==============================] - 0s 169us/sample - loss: 2.0511 - acc: 0.2500 - mean_absolute_error: 0.1723 - mean_squared_error: 0.0846
Epoch 8/15
100/100 [==============================] - 0s 166us/sample - loss: 1.9987 - acc: 0.2800 - mean_absolute_error: 0.1700 - mean_squared_error: 0.0824
Epoch 9/15
100/100 [==============================] 

Epoch 5/15
100/100 [==============================] - 0s 167us/sample - loss: 2.2912 - acc: 0.1000 - mean_absolute_error: 0.1796 - mean_squared_error: 0.0898
Epoch 6/15
100/100 [==============================] - 0s 174us/sample - loss: 2.2828 - acc: 0.1300 - mean_absolute_error: 0.1794 - mean_squared_error: 0.0896
Epoch 7/15
100/100 [==============================] - 0s 203us/sample - loss: 2.2756 - acc: 0.1600 - mean_absolute_error: 0.1792 - mean_squared_error: 0.0894
Epoch 8/15
100/100 [==============================] - 0s 176us/sample - loss: 2.2694 - acc: 0.1700 - mean_absolute_error: 0.1790 - mean_squared_error: 0.0893
Epoch 9/15
100/100 [==============================] - 0s 297us/sample - loss: 2.2640 - acc: 0.1800 - mean_absolute_error: 0.1789 - mean_squared_error: 0.0892
Epoch 10/15
100/100 [==============================] - 0s 175us/sample - loss: 2.2593 - acc: 0.1700 - mean_absolute_error: 0.1787 - mean_squared_error: 0.0891
Epoch 11/15
100/100 [==============================

In [10]:
model.summary

NameError: name 'model' is not defined